In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tweets-sg-100/tweets_sg_100.trainables.syn1neg.npy
/kaggle/input/tweets-sg-100/tweets_sg_100
/kaggle/input/tweets-sg-100/tweets_sg_100.wv.vectors.npy
/kaggle/input/normal22/keras/v1/1/config.json
/kaggle/input/normal22/keras/v1/1/metadata.json
/kaggle/input/normal22/keras/v1/1/model.weights.h5
/kaggle/input/normal/keras/normal/1/sup_tweets200k_v2.h5
/kaggle/input/partiality-scores/labeled.csv
/kaggle/input/partiality-scores/label.csv
/kaggle/input/best_weights/keras/best_best/1/best_weights-2.keras
/kaggle/input/best_weights/keras/best_best/1/best_weights.keras
/kaggle/input/best_weights/keras/best_best/1/best_weights-3.keras
/kaggle/input/best_weights/keras/best_best/1/best_weights-4.keras
/kaggle/input/best_weights/keras/best_best/1/best_weights-8.keras
/kaggle/input/best_weights/keras/best_best/1/best_weights-5.keras
/kaggle/input/best_weights/keras/best_best/1/best_weights-6.keras
/kaggle/input/best_weights/keras/best_best/1/best_weights-9.keras


In [2]:
!pip install gensim==3.4.0
!pip install smart_open==1.9.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 68.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - \ done
  Created wheel for gensim: filename=gensim-3.4.0-cp310-cp310-linux_x86_64.whl size=22474592 sha256=dc67b0b4741d1c2037a6cbd2eef874ac2526d1de682fd13cbffe97c72988b431
  Stored in directory: /root/.cache/pip/wheels/e6/9c/3a/80d40a7717710eaa93a73480bd8ec464c77f45c07491987ed0
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scattertext 0.1.19 requires gensim>=4.0.0, but you have gensim 3.4.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━

# **Pretraining**

In [3]:
from nltk.stem.isri import ISRIStemmer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import re


df = pd.read_csv("/kaggle/input/partiality-scores/label.csv")
sentences = df["preprocessed tweets"].astype(str)
target_sentences = df["preprocessed tweets"].astype(str)

tweets_sg_100 = '/kaggle/input/tweets-sg-100/tweets_sg_100'
sg_model = gensim.models.Word2Vec.load(tweets_sg_100)
stemmer = ISRIStemmer()

# ANALYZE HASHTAGS

ImportError: cannot import name 'Mapping' from 'collections' (/opt/conda/lib/python3.10/collections/__init__.py)

In [ ]:
import re
from nltk.stem.isri import ISRIStemmer
from snowballstemmer import stemmer
ar_stemmer = stemmer("arabic")
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']
    
    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)
    
    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])

    words = text.split()
    for i in range(len(words)):
        words[i] = ar_stemmer.stemWord(words[i])

        
       
    text = " ".join(words)

    return text

In [ ]:
sentences = sentences.apply(clean_str)

In [ ]:
russia = ['روسي','روس','روسيه','لروسي','والروس','ياروسي','روسی','موسك'] # 'روسيا'
belarus = ['يلاروسي','يلاروس','بيلاروس','بيلاروسيه','لبيلاروسي','والبيلاروس'] # 'بيلاروسيا'
ukr = ['اوكران','واوكراني','اوكر','لاوكراني','ناتو'] # 'اوكرانيا'
russian_hashtags = ['جيشالروس','قواتالروسيه','روسياليوم','روسياتنتصر','خارجيهالروسيه','عمليهالعسكريهالروسيه'] # روسيا
ukr_russian_hashtags = ['حربالروسيهالاوكرانيه','روسياوكراني','اوكرانياروسي','غزوالروسيلاوكرانيا','روسياواوكراني'] # 'حرب'
ukr_hashtags = ['روسياتغزواوكراني','روسياتكذب','روسيالص','روسياوالص','روسيامجرم','مجدلاوكرانيا','مخابراتالاوكرانيه'] # 'اوكرانيا'
zelensky = ['زيلينسك','زلينسك'] # 'اوباما'
putin = ['بوتن','وتن','پوتن','وتين','بوتين','پوتين'] #'بوتين'
war = ['حربالعالميه', 'حربالعالميهالثالثه','عسكريه'] # 'حرب'
egypt = ['مصر','مصريه','لمصر'] # 'عربي'
us = ['وامريك','الولاياتالمتحده','الولاياتالمتحده','الولاياتالمتحدهالامريكيه','البريطانيا','والولايا'] # 'امريكا'
gb = ['ريطاني'] # "بريطانيا"

In [ ]:
russia.extend(russian_hashtags)
ukr.extend(ukr_hashtags)
war.extend(ukr_russian_hashtags)

added_words = []
added_words.extend(russia)
added_words.extend(ukr)
added_words.extend(war)
added_words.extend(putin)
added_words.extend(zelensky)
added_words.extend(egypt)
added_words.extend(us)
added_words.extend(gb)
added_words = set(added_words)
russia = set(russia)
ukr = set(ukr)
war = set(war)
putin = set(putin)
zelensky = set(zelensky)
egypt = set(egypt)
us = set(us)
gb = set(gb)

In [ ]:
# add words that donot exist to wv
# russia 'روسيا', ukr 'اوكرانيا', war 'حرب' , putin 'بوتين', zelensky 'اوباما', belarus 'بيلاروسيا' , egypt 'قطر', us 'امريكا', gb "بريطانيا"
for word in russia:
    sg_model.wv.add_vector(word,sg_model.wv['روسيا'])
for word in ukr:
    sg_model.wv.add_vector(word,sg_model.wv['اوكرانيا'])
for word in war:
    sg_model.wv.add_vector(word,sg_model.wv['حرب'])
for word in putin:
    sg_model.wv.add_vector(word,sg_model.wv['بوتين'])
for word in zelensky:
    sg_model.wv.add_vector(word,sg_model.wv['اوباما'])
for word in belarus:
    sg_model.wv.add_vector(word,sg_model.wv['بيلاروسيا'])
for word in egypt:
    sg_model.wv.add_vector(word,sg_model.wv['قطر'])   
for word in us:
    sg_model.wv.add_vector(word,sg_model.wv['امريكا']) 
for word in gb:
    sg_model.wv.add_vector(word,sg_model.wv["بريطانيا"])

In [ ]:
max_len = max([len(sentence.split()) for sentence in sentences])
num_encoder_tokens = max_len
num_decoder_tokens = max_len
print(max_len)
latent_dim = 64
plt.hist([ len(sentence.split()) for sentence in sentences] , bins=range(0,300,2))
plt.show()

In [ ]:
ex_length = 100000
max_len = max([ len(sentence.split()) for sentence in sentences ])
encoder_input_data = np.zeros([ex_length,max_len+5,100])
decoder_output_data = np.zeros([ex_length,max_len+5,100])
decoder_input_data = np.zeros([ex_length,max_len+5,100])

for ex in range(100000):
    words = sentences[ex].split()
    position = 0
    for word in words:
        if word in sg_model.wv:
            encoder_input_data[ex,position,:] = sg_model.wv[word]
            decoder_input_data[ex,position+1,:] = sg_model.wv[word] 
            decoder_output_data[ex,position,:] = sg_model.wv[word] 
        position += 1

In [ ]:
from keras.models import Model
from keras.layers import Input, Bidirectional, LSTM, Dense, Flatten

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 100),name='e_inp') 
decoder_inputs = Input(shape=(None, 100),name='d_inp')
encoder = Bidirectional(LSTM(64, return_state=True),name='enc')
encoder_outputs, hf, hb, cf, cb = encoder(encoder_inputs)
encoder_states = [hf, hb, cf, cb]
decoder_lstm = Bidirectional(LSTM(64, return_sequences=True, return_state=True),name='dec')
decoder_outputs, dhf, dhb, dcf, dcb = decoder_lstm(decoder_inputs,initial_state=encoder_states)
decoder_dense = Dense(100,name="dense_1")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

In [ ]:
from keras.saving import load_model
model = load_model("pretrainig_model.h5")

In [ ]:
model.load_weights("pre_model_bilstm.weights.h5")

In [ ]:
from keras.losses import CosineSimilarity, MeanSquaredError
cs_loss = CosineSimilarity(axis=-1, reduction="sum_over_batch_size", name="cosine_similarity")
model.compile(optimizer='adam',loss=cs_loss ,metrics=['acc'])
model.fit([encoder_input_data, decoder_input_data], decoder_output_data,
          batch_size=16,
          epochs=10,
          validation_split=0.1)


In [ ]:
model.save_weights("pre_model_bilstm.weights.h5")

# **Partiality** 

In [ ]:
df = pd.read_csv("/kaggle/input/partiality-scores/labeled.csv")
df.head()

In [ ]:
features = df['preprocessed tweets'].astype(str)
labels = df['label'].astype(str)

In [ ]:
features_train = list(features[:200000])
features_test = list(features[20000:])
labels_train = list(labels[:200000])
labels_test = list(labels[200000:])

In [ ]:
from nltk.stem.isri import ISRIStemmer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import gensim
import re


tweets_sg_100 = '/kaggle/input/tweets-sg-100/tweets_sg_100'
sg_model = gensim.models.Word2Vec.load(tweets_sg_100)

# ANALYZE HASHTAGS

In [ ]:
# important words not recognized by the embedder 
russia = ['روسي','روس','روسيه','لروسي','والروس','ياروسي','روسی','موسك'] # 'روسيا'
belarus = ['يلاروسي','يلاروس','بيلاروس','بيلاروسيه','لبيلاروسي','والبيلاروس'] # 'بيلاروسيا'
ukr = ['اوكران','واوكراني','اوكر','لاوكراني','ناتو'] # 'اوكرانيا'
russian_hashtags = ['جيشالروس','قواتالروسيه','روسياليوم','روسياتنتصر','خارجيهالروسيه','عمليهالعسكريهالروسيه'] # روسيا
ukr_russian_hashtags = ['حربالروسيهالاوكرانيه','روسياوكراني','اوكرانياروسي','غزوالروسيلاوكرانيا','روسياواوكراني'] # 'حرب'
ukr_hashtags = ['روسياتغزواوكراني','روسياتكذب','روسيالص','روسياوالص','روسيامجرم','مجدلاوكرانيا','مخابراتالاوكرانيه'] # 'اوكرانيا'
zelensky = ['زيلينسك','زلينسك'] # 'اوباما'
putin = ['بوتن','وتن','پوتن','وتين','بوتين','پوتين'] #'بوتين'
war = ['حربالعالميه', 'حربالعالميهالثالثه','عسكريه'] # 'حرب'
egypt = ['مصر','مصريه','لمصر'] # 'عربي'
us = ['وامريك','الولاياتالمتحده','الولاياتالمتحده','الولاياتالمتحدهالامريكيه','البريطانيا','والولايا','امريك'] # 'امريكا'
gb = ['ريطاني'] # "بريطانيا"


russia.extend(russian_hashtags)
ukr.extend(ukr_hashtags)
war.extend(ukr_russian_hashtags)

added_words = []
added_words.extend(russia)
added_words.extend(ukr)
added_words.extend(war)
added_words.extend(putin)
added_words.extend(zelensky)
added_words.extend(egypt)
added_words.extend(us)
added_words.extend(gb)
added_words = set(added_words)
russia = set(russia)
ukr = set(ukr)
war = set(war)
putin = set(putin)
zelensky = set(zelensky)
egypt = set(egypt)
us = set(us)
gb = set(gb)

# add words that donot exist to wv
# russia 'روسيا', ukr 'اوكرانيا', war 'حرب' , putin 'بوتين', zelensky 'اوباما', belarus 'بيلاروسيا' , egypt 'قطر', us 'امريكا', gb "بريطانيا"
# for each set of words assign them to a vector of a similar word to them.
for word in russia:
    sg_model.wv.add_vector(word,sg_model.wv['روسيا'])
for word in ukr:
    sg_model.wv.add_vector(word,sg_model.wv['اوكرانيا'])
for word in war:
    sg_model.wv.add_vector(word,sg_model.wv['حرب'])
for word in putin:
    sg_model.wv.add_vector(word,sg_model.wv['بوتين'])
for word in zelensky:
    sg_model.wv.add_vector(word,sg_model.wv['اوباما'])
for word in belarus:
    sg_model.wv.add_vector(word,sg_model.wv['بيلاروسيا'])
for word in egypt:
    sg_model.wv.add_vector(word,sg_model.wv['قطر'])   
for word in us:
    sg_model.wv.add_vector(word,sg_model.wv['امريكا']) 
for word in gb:
    sg_model.wv.add_vector(word,sg_model.wv["بريطانيا"])

In [ ]:
# how many tweets 
ex_length = 200000
# how many words per tweet : pad with zeros if words are less than max_len
max_len = 64
# each word is represented in a vector of size 100 
word_vector = 100
X_train = np.zeros([ex_length,max_len,word_vector])

In [ ]:
for ex in range(200000):
    words = features[ex].split()
    position = 0
    for word in words:
        if word in sg_model.wv:
            X_train[ex,position,:] = sg_model.wv[word]
        position += 1

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Map labels to integer values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(labels_train[:200000])

y_train_one_hot = to_categorical(y_train_encoded)

In [ ]:
encoder = model.get_layer('enc')
config = encoder.get_config()
config['layer']['config']['return_state'] = False
config['layer']['config']['return_sequences'] = True

encoder_layer_new = Bidirectional(LSTM.from_config(config['layer']['config']), name='enc')

# Get the weights of the original encoder layer and set them to the new layer
encoder_layer_new.build((None,None,100))
encoder_layer_new.set_weights(encoder.get_weights())       

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense
from tensorflow.keras.losses import CategoricalCrossentropy

# Define the model
model = Sequential()
# Add Bidirectional LSTM layers
model.add(encoder_layer_new)
model.add(Bidirectional(LSTM(units=128, return_sequences=False)))

# Add Dense layer
model.add(Dense(units=20, activation='relu'))

# Add Softmax layer
model.add(Dense(units=3, activation='softmax'))

loss= CategoricalCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction="sum_over_batch_size",
    name="categorical_crossentropy",
)
# Compile the model
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

# Print model summary
model.summary()


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint filepath with '.keras' extension
checkpoint_filepath = 'best_weights.keras'

# Define the ModelCheckpoint callback
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,
                                      save_best_only=True,
                                      monitor='val_accuracy',
                                      mode='max',  # 'max' for accuracy
                                      verbose=1)

# Train the model with ModelCheckpoint callback
history = model.fit(X_train, y_train_one_hot,
                    epochs=1,
                    batch_size=32,
                    validation_split=0.1,
                    callbacks=[checkpoint_callback],shuffle = True)


In [ ]:
test_set = list(sentences[200000:].astype(str))
test_set[0]

In [ ]:
test_set = list(sentences[200000:].astype(str))
ex_length = len(test_set)
encoder_input_data = np.zeros([ex_length,64,100])
for ex in range(ex_length):
    words = test_set[ex].split()
    position = 0
    for word in words:
        if word in sg_model.wv: 
            encoder_input_data[ex,position,:] = sg_model.wv[word]
        position += 1

In [ ]:
y_test_encoded = label_encoder.fit_transform(labels_test)
y_test_one_hot = to_categorical(y_test_encoded)

In [ ]:
loss,accuracy = model.evaluate(encoder_input_data,y_test_one_hot)

print(f"Loss :{loss:.4f}")
print(f"Accuracy :{accuracy:.4f}")

In [ ]:
model.save_weights("post_train.h5")

In [ ]:
model.load_weights("/kaggle/input/best_weights/keras/best_best/1/best_weights-9.keras")

In [ ]:
loss,accuracy = model.evaluate(encoder_input_data,y_test_one_hot)

print(f"Loss :{loss:.4f}")
print(f"Accuracy :{accuracy:.4f}")

In [ ]:
model.load_weights("/kaggle/input/normal22/keras/v1/1/model.weights.h5")

In [ ]:
# model 4 is the best model
# model.weights.h5